In [11]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [12]:
df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases'
                 + '/iris/iris.data', header=None)


df = df.sample(frac=1) # shuffle

# add label indices column , i.e convert flower label to number
mapping = {k: v for v, k in enumerate(df[4].unique())}
df[5] = df[4].map(mapping)


# normalise the data
alldata = torch.tensor(df.iloc[:, [0,1,2,3]].values, dtype=torch.float)
alldata = (alldata - alldata.mean(dim=0)) / alldata.var(dim=0)

# create datasets

target_tr = torch.tensor(df.iloc[:100, 5].values, dtype=torch.long)
target_va = torch.tensor(df.iloc[100:, 5].values, dtype=torch.long)
data_tr = alldata[:100]
data_va = alldata[100:]



In [34]:

def mlp(X, y, lr=0.01, num_epochs = 100):
    
    W1 = torch.randn(4,12, requires_grad=True)
    W2 = torch.randn(12,3, requires_grad=True)
    b1 = torch.tensor(0.0, requires_grad=True)
    b2 = torch.tensor(0.0, requires_grad=True)
    
    for x in range(num_epochs):
        
        W1.grad = None
        W2.grad = None
        b1.grad = None
        b2.grad = None
        
        logits = torch.relu(X @ W1 + b1) @ W2 + b2
        error = torch.nn.functional.cross_entropy(logits, y)
        #print(f"Error  at {x} is {error}")
        error.backward()
        
        w1 = W1.data - W1.grad*lr
        W1.data = w1
        
        w2 = W2.data - W2.grad*lr
        W2.data = w2
        
        B1 = b1 - b1.grad*lr
        b1.data = B1
        B2 = b2 - b2.grad*lr
        b2.data = B2
    
    print(f"Training Error is {error}")
    return  W1, W2, b1, b2, error
    

In [35]:
W1, W2, b1, b2, error = mlp(data_tr, target_tr)
logits = torch.relu(data_va @ W1 + b1) @ W2 + b2
error = torch.nn.functional.cross_entropy(logits, target_va)
print(f"Validation Error is {error}")

Training Error is 0.5068268179893494
Validation Error is 0.6615543365478516


In [41]:

def getAccuracy(logits, targets):
    
    _, predicted_tr = torch.max(logits.data, 1)
    accuracy_tr = (predicted_tr == targets).sum().data.numpy()/predicted_tr.shape
    return accuracy_tr
    
    # predictions = []
    # for i in range(logits.shape[0]):
    #     _, index = torch.max(logits[i],0)
    #     predictions.append(index)
    
    #score = accuracy_score(targets, predictions)
    
    #return score

In [37]:
%matplotlib qt
train_error = np.zeros(10)
val_error = np.zeros(10)

for x in range(10):
    W1, W2, b1, b2, error = mlp(data_tr, target_tr)
    train_error[x] = error
    logits = torch.relu(data_va @ W1 + b1) @ W2 + b2
    error = torch.nn.functional.cross_entropy(logits, target_va)
    val_error[x] = error

x = np.linspace(1,10,10)
plt.plot(x, train_error, label="training error") 
plt.plot(x, val_error, label="validation error")
plt.legend()

Training Error is 0.6117843389511108
Training Error is 0.649422824382782
Training Error is 0.4925763010978699
Training Error is 0.5316982865333557
Training Error is 0.6342013478279114
Training Error is 0.7649069428443909
Training Error is 0.5610867142677307
Training Error is 0.6748521327972412
Training Error is 0.45868685841560364
Training Error is 0.5051885843276978


In [32]:
W1, W2, b1, b2 ,_ = mlp(data_tr, target_tr)
logits = torch.relu(data_tr @ W1 + b1) @ W2 + b2
score  = getAccuracy(logits, target_tr)
print(score)

Training Error is 0.43027064204216003
0.85


In [46]:

%matplotlib qt
train_score = np.zeros(10)
val_score = np.zeros(10)

for x in range(10):
    W1, W2, b1, b2, error = mlp(data_tr, target_tr)
    
    logits = torch.relu(data_tr @ W1 + b1) @ W2 + b2
    score = getAccuracy(logits, target_tr)
    train_score[x] = score
    
    logits_2 = torch.relu(data_va @ W1 + b1) @ W2 + b2
    score = getAccuracy(logits_2, target_va)
    val_score[x] = score

x = np.linspace(1,10,10)
plt.plot(x, train_score, label="Training Accuracy") 
plt.plot(x, train_error, label="Validation Accuracy")
plt.legend()
plt.savefig("MLP_accuracy.png")

Training Error is 0.4551851153373718
Training Error is 0.7915902137756348
Training Error is 0.4932372272014618
Training Error is 0.7178360819816589
Training Error is 0.537736713886261
Training Error is 0.3295105993747711
Training Error is 0.7705748677253723
Training Error is 0.6578662395477295
Training Error is 0.3412044942378998
Training Error is 0.5136879086494446


In [19]:
# def MLP_2(data_tr, data_va, targets_tr, targets_va, epochs = 100, lr = 0.01):
#   W1 = torch.randn((4,12), requires_grad=True, dtype= torch.float)
#   W2 = torch.randn((12,3), requires_grad=True, dtype= torch.float)
#   b1 = torch.tensor(0.0, requires_grad=True)
#   b2 = torch.tensor(0.0, requires_grad=True)
# 
#   for _ in range(epochs):
#     logits = torch.relu(data_tr @ W1 + b1) @ W2 + b2
#     loss = torch.nn.functional.cross_entropy(logits, targets_tr)
#     loss.backward()
# 
#     z_W1 = W1 - W1.grad*lr
#     z_W2 = W2 - W2.grad*lr
#     z_b1 = b1 - b1.grad*lr
#     z_b2 = b2 - b2.grad*lr
# 
#     W1.data = z_W1
#     W2.data = z_W2
#     b1.data = z_b1
#     b2.data = z_b2
#       
#     #Clears old gradients
#     W1.grad.detach().zero_()
#     W2.grad.detach().zero_()
#     b1.grad.detach().zero_()
#     b2.grad.detach().zero_()
# 
# 
#   W1, W2, b1, b2 ,_ = mlp(data_tr, target_tr) 
# 
#   predicted = torch.relu(data_tr @ W1 + b1) @ W2 + b2
#   #_, predicted_tr = torch.max(predicted.data, 1)
#   #accuracy_tr = (predicted_tr == targets_tr).sum().data.numpy()/predicted_tr.shape
#   score = getAccuracy(predicted,targets_tr)
#   print(score)
# 
# 
#   predicted2 = torch.relu(data_va @ W1 + b1) @ W2 + b2
#   #_, predicted_va = torch.max(predicted2.data, 1)
#   #accuracy_va = (predicted_va == targets_va).sum().data.numpy()/predicted_va.shape
#   score_2 = getAccuracy(predicted2,targets_va)
#   print(score_2)
# 
#   return score, score_2


In [20]:

# train_acc = np.empty(20)
# test_acc = np.empty(20)
# iterations = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
# 
# for i in range(20):
#   accuracy_tr, accuracy_va = MLP_2(data_tr, data_va, target_tr, target_va)
#   train_acc[i] = accuracy_tr
#   test_acc[i] = accuracy_va
# 
# plt.plot(iterations, train_acc, label = "Train")
# plt.plot(iterations, test_acc, label = "Validation")
# plt.xlabel("Run number")
# plt.ylabel("Accuracy")
# #plt.xticks(np.arange(1, 21, step=1))
# plt.legend()
# plt.title("Accuracy on validation and train data")